In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

In [10]:
CLIENT_ID = 'A3XLANGDCEGKOVCYZG0YCEVSGHBVGKXVBIATRIJOBNZ4LLDP' #Foursquare ID
CLIENT_SECRET = 'TDJZCZSW5KHSJXMRVUYCMUITWDIXU3DKJ0WA0V03FVWEFCAU' #Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [11]:

LIMIT = 500 
cities = ["London, UK", 'Cambridge, UK', 'Oxford, UK', 'Edinburgh, UK', 'Manchester, UK']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d16d941735") # CAFE CATEGORY ID
    results[city] = requests.get(url).json()

In [13]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [33]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of cafe in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of cafe in London, UK =  332
Showing Top 100
Total number of cafe in Cambridge, UK =  67
Showing Top 100
Total number of cafe in Oxford, UK =  105
Showing Top 100
Total number of cafe in Edinburgh, UK =  223
Showing Top 100
Total number of cafe in Manchester, UK =  165
Showing Top 100


In [17]:
maps[cities[0]] #Map of london


In [20]:
maps[cities[1]] #Map of cambridge

In [22]:
maps[cities[2]] #Map of Oxford

In [24]:
maps[cities[3]] #Map of Edinburgh

In [26]:
maps[cities[4]] #Map of Manchester

In [27]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

London, UK
Mean Distance from Mean coordinates
0.040320640819730996
Cambridge, UK
Mean Distance from Mean coordinates
0.017165931042392914
Oxford, UK
Mean Distance from Mean coordinates
0.01950187164271548
Edinburgh, UK
Mean Distance from Mean coordinates
0.021320584073764873
Manchester, UK
Mean Distance from Mean coordinates
0.0413666806675109


In [28]:
maps[cities[0]] #Map of london

In [29]:
maps[cities[1]] #Map of cambridge

In [30]:
maps[cities[2]] #Map of Oxford

In [31]:
maps[cities[3]] #Map of Edinburgh

In [32]:
maps[cities[4]] #Map of Manchester